In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def url_get():
    #url = input("请输入要爬取的首页url:")
    url='https://blog.csdn.net/qq1332479771/article/details/62104624'
    try:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36'}
        requests.get(url, headers=headers)
        return url
    except:
        print("url无法连接")
    return url_get()

In [15]:
def spiderpage(url,x):
    kv = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) Chrome/57.0.2987.98 Safari/537.36 LBBROWSER'}
    r = requests.get(url, headers=kv)
    r.encoding = r.apparent_encoding
    pagetext = r.text
    pagelinks = re.findall(r'(?<=<a href=\").*?(?=\")|(?<=href=\').*?(?=\')', pagetext)
    bsObj=BeautifulSoup(pagetext)
    link=bsObj.body.a.text
     
    #page_list = response.xpath('//div[@class="l_post l_post_bright j_l_post clearfix  "]')
    
    title=bsObj.body.h1.text
    path='C:\\Users\\22478\\Desktop\\data\\'
    fileName=path+str(x)+'.txt'
    file = open(fileName, 'a', encoding='utf8')
    file.write(f'{url}\n')
    file.write(f'{pagelinks}\n')
    file.write(f'{title}\n')
    file.write(f'{pagetext}\n')
    file.close()
    return pagelinks

In [7]:
def url_filtrate(pagelinks):
    # 去除不是以要爬取网站开头的url,如跳转的广告等
    same_target_url = []
    for l in pagelinks:
        if re.findall(r'blog.csdn.net/\w+/article/details/\d+', l):
            # 根据对网页的分析添加筛选条件,过滤掉系统推荐的博文链接
            if re.findall(r'blockchain_lemon', l):
                pass
            # 过滤掉广告链接
            elif re.findall(r'passport', l):
                pass
            else:
                same_target_url.append(l)
    # 去除重复url
    unrepect_url = []
    for l in same_target_url:
        if l not in unrepect_url:
            unrepect_url.append(l)
    return unrepect_url

In [8]:
class linkQuence:
    def __init__(self):
        # 已访问的url集合
        self.visited = []
        # 待访问的url集合
        self.unvisited = []

    # 获取访问过的url队列
    def getvisitedurl(self):
        return self.visited

    # 获取未访问的url队列
    def getunvisitedurl(self):
        return self.unvisited

    # 添加url到访问过得队列中
    def addvisitedurl(self, url):
        return self.visited.append(url)

    # 移除访问过得url
    def removevisitedurl(self, url):
        return self.visited.remove(url)

    # 从未访问队列中取一个url
    def unvisitedurldequence(self):
        try:
            return self.unvisited.pop()
        except:
            return None

    # 添加url到未访问的队列中
    def addunvisitedurl(self, url):
        if url != "" and url not in self.visited and url not in self.unvisited:
            return self.unvisited.insert(0, url)

    # 获得已访问的url数目
    def getvisitedurlount(self):
        return len(self.visited)

    # 获得未访问的url数目
    def getunvistedurlcount(self):
        return len(self.unvisited)

    # 判断未访问的url队列是否为空
    def unvisitedurlsempty(self):
        return len(self.unvisited) == 0

In [9]:
class Spider():
    def __init__(self, url):
        self.linkQuence = linkQuence()  # 将队列引入本类
        self.linkQuence.addunvisitedurl(url)  # 传入待爬取的url,即爬虫入口

    def crawler(self,urlcount):
        # 子页面过多,为测试方便加入循环控制子页面数量
        x = 1
        while x <= urlcount:
            # 若子页面不是很多,可以直接使用队列中的未访问列表非空作为循环条件
            # while not self.linkQuence.unvisitedurlsempty():
            if x > 1:
                print(f"第{x-1}个url,开始爬")
            visitedurl = self.linkQuence.unvisitedurldequence()  # 从未访问列表中pop出一个url
            if visitedurl is None or visitedurl == '':
                continue
            initial_links = spiderpage(visitedurl,x)  # 爬出该url页面中所有的链接
            right_links = url_filtrate(initial_links)  # 筛选出合格的链接
            self.linkQuence.addvisitedurl(visitedurl)  # 将该url放到访问过的url队列中
            for link in right_links:  # 将筛选出的链接放到未访问队列中
                self.linkQuence.addunvisitedurl(link)
            x += 1
        print(f"终于爬完了,一共是{x-2}个url")
        return self.linkQuence.visited

In [10]:
def writetofile(list):
    # 因为第一个爬取的页面为爬虫入口,非需要的博文网址,因此从[1]开始写入
    x=1
    for url in list[1:]:
        # urls.txt用于保存博文标题和博文链接,文件夹demo创建好,或者加入os也行,反正很简单
        file = open('E:/urls.txt', 'a', encoding='utf8')
        file.write(f'{url}\n')
        x += 1
    file.close()
    print(f'写入已完成,总计{x-1}个子链接')

In [16]:
if __name__ == '__main__':
    url = url_get()
    spider = Spider(url)
    #传入要爬取的子链接数量100
    urllist = spider.crawler(500)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


第1个url,开始爬
第2个url,开始爬
第3个url,开始爬
第4个url,开始爬
第5个url,开始爬
第6个url,开始爬
第7个url,开始爬
第8个url,开始爬
第9个url,开始爬
第10个url,开始爬
第11个url,开始爬
第12个url,开始爬
第13个url,开始爬
第14个url,开始爬
第15个url,开始爬
第16个url,开始爬
第17个url,开始爬
第18个url,开始爬
第19个url,开始爬
第20个url,开始爬
第21个url,开始爬
第22个url,开始爬
第23个url,开始爬
第24个url,开始爬
第25个url,开始爬
第26个url,开始爬
第27个url,开始爬
第28个url,开始爬
第29个url,开始爬
第30个url,开始爬
第31个url,开始爬
第32个url,开始爬
第33个url,开始爬
第34个url,开始爬
第35个url,开始爬
第36个url,开始爬
第37个url,开始爬
第38个url,开始爬
第39个url,开始爬
第40个url,开始爬
第41个url,开始爬
第42个url,开始爬
第43个url,开始爬
第44个url,开始爬
第45个url,开始爬
第46个url,开始爬
第47个url,开始爬
第48个url,开始爬
第49个url,开始爬
第50个url,开始爬
第51个url,开始爬
第52个url,开始爬
第53个url,开始爬
第54个url,开始爬
第55个url,开始爬
第56个url,开始爬
第57个url,开始爬
第58个url,开始爬
第59个url,开始爬
第60个url,开始爬
第61个url,开始爬
第62个url,开始爬
第63个url,开始爬
第64个url,开始爬
第65个url,开始爬
第66个url,开始爬
第67个url,开始爬
第68个url,开始爬
第69个url,开始爬
第70个url,开始爬
第71个url,开始爬
第72个url,开始爬
第73个url,开始爬
第74个url,开始爬
第75个url,开始爬
第76个url,开始爬
第77个url,开始爬
第78个url,开始爬
第79个url,开始爬
第80个url,开始爬
第81个url,开始爬
第82个url,开始爬
第83个url,开始爬
第84个url,开始爬
第